# Simple VisionTransformer

2023/10/27 M.Udagawa

google colabを想定

内容は「Transformer variational wave functions for frustrated quantum spin systems(arXiv:2211.05504v2)」を参照

Attentionの理解にはこれで十分かもしれないが、本来のViTとしては位置エンコーダやMLPらへんは省略されている点は注意

また現状(2023/10/27)、**このモデルを使ってもボースハバード模型の基底状態は求まらない**。あくまで、Attentionの理解というスタンスで使ってください

次の初学者向けのタブは長いので、左端の▽で閉じるべし。

# 以下は初学者向け

## 想定する読者のレベル


内容としては以下がわかれば理解できる

* ベクトルの内積、行列の積
* Pythonでforループができる
* Pytorchで全結合ニューラルネットワークが作れる
* numpyの操作(reshape, random)
* ソフトマックス関数

・（ちょい難）ベクトルや行列の計算を成分でできるならAttentionは（数学的に）即座に理解できる->興味がある人は「アインシュタインの縮約記法」で検索、これができるようになると解析力学、ベクトル解析、量子力学などで圧倒的成長を感じられる。なにより便利である。


## コード上のnumとは？


詳細はボースハバード模型のコードを参照

numは(サンプル,格子点)を表している。なので

num[i]

は$i$番目のサンプルを意味するベクトルだし、

num[i][j]

は$i$番目のサンプルの格子点$j$の値を意味する。値とは、例えば、Ising模型ではスピン$\{\pm1\}$、ボースハバード模型ではその格子の粒子数となる。



## 論文中ではANNの入力は１つのサンプル=ベクトルになっていたがコード中は全サンプル=行列を代入している。これでいいのか？



問題ない、ようにANNを作ればよい。もちろんベクトルを入力とするようなANNを作って利用することも可能だが、

*   コードの行数削減
*   計算速度の短縮
*   （最重要）動くけど計算は間違いのとき、バグがマジで見つからない

といった理由で行列にして明示的に操作する方針を取っている。コードを1行ずつかみ砕いていけば、実際は各サンプルごとに計算しているのがわかるはず。

# セットアップ

ライブラリのインポート

einopsはcolabの初期環境には存在しないため、ランタイム接続時にインストールする必要がある。
文頭の!は「これからLinuxコマンドを打つ」宣言に相当する

In [1]:
import numpy as np
import torch
import torch.nn as nn
#!pip install einops
from einops.layers.torch import Rearrange

初期値の設定

下部ではTransformer内部でどんな値になっているか確認できるが、NGが表示されている場合は使えない

In [15]:
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# n_sample:サンプル数
# n_site:サイトの数
# num:サンプル(n_sample, n_site)
# patch_size:パッチサイズ
# n_heads:ヘッドの数
# depth:ViTの層の深さ(Attentionのループの回数)
#
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

n_sample = 2
n_site = 8
num = np.ones((n_sample, n_site))
patch_size = 2
n_heads = 2
depth = 1

print(num)

print('---check---')
if n_site % patch_size == 0:
  print('number of patches: ', n_site//patch_size)
else:
  print('patch size: NG')

if patch_size % n_heads == 0:
  print('dimension per head: ', patch_size//n_heads)
else:
  print('number of heads: NG')

[[1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1.]]
---check---
number of patches:  4
dimension per head:  1


デバイスの設定

In [16]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cpu


# パッチ分割

インプットとなる粒子数の配置は次のような形になっている

$ input = \begin{bmatrix} \mathbf{n}_1 = [0, 2, \dots ,1] \\ \mathbf{n}_2  = [1, 1, \dots 0]\\ \vdots \\ \mathbf{n}_{{n_{sample}}}=[2,1,\dots 1] \end{bmatrix}$

$input.shape = [n_{sample}, n_{site}]$

この中から一つのサンプルを取り出してみる、例えば$n_{site}=4$なら

$\mathbf{n}_i = [2,0,1,1]$

パッチ分割というのは、これを等分していくつかのベクトルに分けるということである。例えばパッチサイズが2なら

$\mathbf{n_i} -> [2, 0], [1, 1] $

これを全てのサンプルに対して行うと

$input.shape \rightarrow x.shape= [n_{sample},n_{patches}, patch\ size]$

ただし、$n_{pathces}$は$n_{site} = patch\ size \times n_{patches}$ をみたす正整数でなければならない。

自然言語処理でいうならばこの一つのパッチが単語に相当しており、それゆえパッチ分割は後述のAttentionのために必須の操作である。

また、torch.tensorの配列の操作として、einopsライブラリのRearrangeを使うと便利らしく、実際以下では頻繁に利用する（torch.reshapeでも対応できるかはやってみないと分からないが、現時点(2023/10/20)でRearrangeに不満はない）。

In [17]:
class Patching(nn.Module):
  def __init__(self, patch_size):
    super().__init__()

    # inputのshapeは(n_sample, n_site)
    # n_site = patch_size * n_patches (definition of n_patches(int))
    # そこで次の変換をする
    # (n_sample, n_site) -> (n_sample, n_patches, patch_size)
    # s:n_sample, np:n_patches, ps:patch_size

    self.new_input = Rearrange('s (np ps) ->s np ps', ps=patch_size)

  def forward(self, x):
    output = self.new_input(x)
    return output

動かしてみる。

In [18]:
num = np.ones((n_sample, n_site))
model = Patching(patch_size)
num = model(num)
print(num)

[[[1. 1.]
  [1. 1.]
  [1. 1.]
  [1. 1.]]

 [[1. 1.]
  [1. 1.]
  [1. 1.]
  [1. 1.]]]


# アテンション機構

早速本題である。以下では、簡単のため議論上はサンプル数1として考える(コードは$n_{sample}$)。

パッチ分割により、この部分への入力は

$x.shape = [n_{patches}, patch\ size] $

となっている。すなわち、粒子数分布(１個の$n_{site}$次元ベクトル)を$n_{patches}$個の$patch\ size$次元ベクトルに分割されている。

$\mathbf{n}_i = [2,0,1,1] -> [2, 0], [1, 1] = \mathbf{x_1}, \mathbf{x_2}$

この一つ一つのベクトルから、さらに3個ずつベクトルを生成する。

$\mathbf{q}_i = W_q\mathbf{x}_i$

$\mathbf{k}_i = W_k\mathbf{x}_i$

$\mathbf{v}_i = W_v\mathbf{x}_i, (W_\nu.shape = [patch\ size, patch\ size],\mu = q,k,v)$

なお重み$W_\mu$は一般的に全結合ネットワーク一層に相当する。

これらは順にクエリquery、キーkey、バリューvalueという。これらによって、アテンションは次のように定義される

$\mathbf{A}_i = \sum_j Softmax(\frac{\mathbf{q}_i\cdot \mathbf{k}_j}{\sqrt{d}}) \mathbf{v}_j$

一つ一つ順を追って説明しよう。まず、ソフトマックスの中にある内積に注目する。

$\mathbf{q}_i\cdot\mathbf{k}_j$

これらのベクトルの添え字はもともとパッチの場所を指定していたことを思い出そう。するとこれは、パッチとパッチの間の関係、すなわち長距離相関を捉えている(逆に一つ一つのベクトルq,k,vはあるパッチの情報、すなわち局所的な特徴を捉えている)。

とはいえこの時点では二つのパッチ（2単語）の関係しか言えない。うまいこと系全体（文章）の特徴を捉えた量に変換出来ないだろうか。

ここでもう一度添え字に着目すると内積$\mathbf{q}_i\cdot\mathbf{k}_j$は二つの添え字$(i,j)$で定まる。これをもう少し俯瞰してみる。すなわち、二つの添え字$(i,j)$で定まるものといえば行列の成分があったことを思い出す。このような異なる添え字のついたもの同士の積は一般的に行列として扱う（当然添え字が3つ以上も可能、その場合は多次元配列となる）。

この行列とすべてのバリュー$\mathbf{v}_i$を使えば、系全体(文章)の特徴を取り入れた新しいパッチ(単語の抽象版?)を$n_{patches}$個作れそうである。

$\mathbf{A}_i \overset{?}{=} \sum_j (\mathbf{q}_i\cdot\mathbf{k}_j) \mathbf{v}_j$

ただしこのままの定義だと行列の要素が極端に大きくなり特定のパッチからの影響を強く受ける可能性がある。だからソフトマックスを使って0以上1以下の数値に規格化している。なお、ソフトマックス関数の規格化は$\mathbf{k}_j$の添え字$j$についてとる。この部分の構造は頻繁に使われるため、Attention weight $\alpha_{ij}$としてよく書かれる。

$\alpha_{ij} = Softmax(\frac{\mathbf{q}_i\cdot\mathbf{k}_j}{\sqrt{d}})$

また、アテンションは入力と全く同じ構造をしている。

$[\mathbf{x}_i].shape = [\mathbf{A}_i].shape = [n_{patches}, patch\ size]$

そのため、出力を再度入力としてループさせることが可能である。本来のViTでは残差接続を活用して深層化しているようだ。

以上がアテンションの概要である。

実際のプログラムではheadというものを使っているが、本質的にはあまり変わらない（イメージ的にはパッチの中の同じ部分ごとにAttentionを計算している）。Rearrangeを最初と最後にしているだけなので、コードを見て理解するのが速いだろう。

In [19]:
class Attention(nn.Module):
  def __init__(self, patch_size, n_heads):
    super().__init__()
    #
    # input(n_sample, n_patches, patch_size)
    # self.dim_heads:一つのヘッドに与えられるベクトルの長さ
    #
    self.n_heads = n_heads
    self.dim_heads = patch_size//n_heads

    super().__init__()

    self.W_Q = nn.Linear(patch_size, patch_size, bias=False)
    self.W_K = nn.Linear(patch_size, patch_size, bias=False)
    self.W_V = nn.Linear(patch_size, patch_size, bias=False)

    self.split_into_heads = Rearrange("s np (nh dh) -> s nh np dh", nh = self.n_heads)

    self.softmax = nn.Softmax(dim = -1)

    self.concat = Rearrange("s nh np dh -> s np (nh dh)", nh = self.n_heads)

  def forward(self, x):
    #
    # x == input:(n_sample, n_patches, patch_size)
    # q,k,v != input:(n_sample, n_patches, patch_size)
    # q,k,v in head:(n_sample, n_heads, n_patches, dim_heads)
    # k.transpose(-1, -2):(n_sample, n_heads, dim_heads, n_patches)
    # matmulは第一引数の最後尾と第二引数の最後尾から２個目のサフィックスの行列積
    # logit, attention_weight:(n_sample, n_heads, n_patches, n_patches)
    # output before concat:(n_sample, n_heads, n_patches, dim_heads)
    # output :(n_sample, n_patches, patch_size)
    #
    q = self.W_Q(x)
    k = self.W_K(x)
    v = self.W_V(x)

    q = self.split_into_heads(q)
    k = self.split_into_heads(k)
    v = self.split_into_heads(v)

    logit = torch.matmul(q, k.transpose(-1, -2)) * (self.dim_heads ** -0.5)
    attention_weight = self.softmax(logit)

    output = torch.matmul(attention_weight, v)
    output = self.concat(output)

    return output

## torch.matmulの確認

まずはベクトルから

$ \mathbf{q}\cdot\mathbf{k} = \sum_i q_ik_i$

In [20]:
#確認用(check)
q_ = torch.arange(3)
k_ = torch.arange(3)
print('q=', q_, q_.shape)
print('k=', k_, k_.shape)
print('torch.matmul(q, k) = ',torch.matmul(q_, k_))
print('torch.matmul(k, q) = ',torch.matmul(k_, q_))

q= tensor([0, 1, 2]) torch.Size([3])
k= tensor([0, 1, 2]) torch.Size([3])
torch.matmul(q, k) =  tensor(5)
torch.matmul(k, q) =  tensor(5)


$Q= \begin{pmatrix}1&0&0 \\ 0&1&0\end{pmatrix}, K = \begin{pmatrix}0& 1&2 \\ 3 & 4&5\end{pmatrix}$

$Q.shape=[2\times3], K.shape=[2\times3]$

In [21]:
Q_ = torch.tensor([[1,0,0],[0,1,0]])
print("Q=", Q_, Q_.shape)
K_ = torch.arange(6)
K_ = torch.reshape(K_, (2,3))
print("K=",K_, K_.shape)

Q= tensor([[1, 0, 0],
        [0, 1, 0]]) torch.Size([2, 3])
K= tensor([[0, 1, 2],
        [3, 4, 5]]) torch.Size([2, 3])


In [22]:
print(torch.matmul(Q_, K_))

RuntimeError: mat1 and mat2 shapes cannot be multiplied (2x3 and 2x3)

$K^T.shape=[3\times2]$

In [23]:
print(torch.matmul(Q_, K_.T))

tensor([[0, 3],
        [1, 4]])


In [24]:
print(K_.T)
print(K_.transpose(0, 1))
print(K_.transpose(1, 0))
#transposeの引数は転置したい二つのaxisを選んでいる
#順番は関係ない？
print(K_.transpose(-1, -2))
print(K_.transpose(-2, -1))
#dimの指定におけるマイナスは、最後尾から何番目の配列を表す。
#2次元配列の場合は,[0=-2], [1=-1]

tensor([[0, 3],
        [1, 4],
        [2, 5]])
tensor([[0, 3],
        [1, 4],
        [2, 5]])
tensor([[0, 3],
        [1, 4],
        [2, 5]])
tensor([[0, 3],
        [1, 4],
        [2, 5]])
tensor([[0, 3],
        [1, 4],
        [2, 5]])


ここから本題

$q.shape=k.shape=[n_{sample}, n_{heads}, n_{patch}, dim_{head}]$

headの中の話だから、前二つは無関係

In [25]:
#確認用(check)
q_ = torch.arange(n_sample*n_heads*(n_site//patch_size)*(patch_size//n_heads))
q_ = q_.reshape((n_sample, n_heads, n_site//patch_size, patch_size//n_heads))
print('q=', q_.shape)
k_ = torch.arange(n_sample*n_heads*(n_site//patch_size)*(patch_size//n_heads))
k_ = k_.reshape((n_sample, n_heads, n_site//patch_size, patch_size//n_heads))
print('k=', k_.shape)


q= torch.Size([2, 2, 4, 1])
k= torch.Size([2, 2, 4, 1])


In [26]:
print('k.transpose=',k_.transpose(-1,-2).shape)
logit_ = torch.matmul(q_, k_.transpose(-1,-2))
print('logit=',logit_, logit_.shape)
output_ = torch.matmul(logit_, q_)
print('output=', output_, output_.shape)

k.transpose= torch.Size([2, 2, 1, 4])
logit= tensor([[[[  0,   0,   0,   0],
          [  0,   1,   2,   3],
          [  0,   2,   4,   6],
          [  0,   3,   6,   9]],

         [[ 16,  20,  24,  28],
          [ 20,  25,  30,  35],
          [ 24,  30,  36,  42],
          [ 28,  35,  42,  49]]],


        [[[ 64,  72,  80,  88],
          [ 72,  81,  90,  99],
          [ 80,  90, 100, 110],
          [ 88,  99, 110, 121]],

         [[144, 156, 168, 180],
          [156, 169, 182, 195],
          [168, 182, 196, 210],
          [180, 195, 210, 225]]]]) torch.Size([2, 2, 4, 4])
output= tensor([[[[    0],
          [   14],
          [   28],
          [   42]],

         [[  504],
          [  630],
          [  756],
          [  882]]],


        [[[ 2928],
          [ 3294],
          [ 3660],
          [ 4026]],

         [[ 8808],
          [ 9542],
          [10276],
          [11010]]]]) torch.Size([2, 2, 4, 1])


## 実際に動かす

流れを確認するため、それまでのクラスをすべて使う

In [27]:
num = np.ones((n_sample, n_site))
model = Patching(patch_size)
model = model.to(device)
num = model(num)

model = Attention(patch_size, n_heads)
model = model.to(device)
num = torch.from_numpy(num.astype(np.float32)).detach().to(device)
num = model(num)
num = num.to('cpu').detach().numpy().copy()
print(num)
#変換時の末尾のclone(),copy()について
#numpyとtorch.tensorはメモリを共有しているため、明示的に書いておくべし

[[[0.61085075 0.67732286]
  [0.61085075 0.67732286]
  [0.61085075 0.67732286]
  [0.61085075 0.67732286]]

 [[0.61085075 0.67732286]
  [0.61085075 0.67732286]
  [0.61085075 0.67732286]
  [0.61085075 0.67732286]]]


# Transformer Encoder

基本的には以下のループ

Attention -> 全結合層 -> 活性化関数

In [28]:
class Encoder(nn.Module):
  def __init__(self, depth, n_heads):
    super().__init__()
    self.depth = depth

    self.Attention = Attention(patch_size, n_heads)

    self.W = nn.Linear(patch_size, patch_size, bias=False)

    #self.nonlinear = nn.functional.relu()

  def forward(self, x):

    for _ in range(self.depth):
      output = self.Attention(x)
      output = self.W(output)
      output = nn.functional.relu(output)

    return output

In [29]:
num = np.ones((n_sample, n_site))
model = Patching(patch_size)
model = model.to(device)
num = model(num)

###ここから
model = Encoder(depth, n_heads)
model = model.to(device)
num = torch.from_numpy(num.astype(np.float32)).detach().to(device)
num = model(num)
num = num.to('cpu').detach().numpy().copy()
print(num.shape)
print(num)

(2, 4, 2)
[[[0.05886333 0.        ]
  [0.05886333 0.        ]
  [0.05886333 0.        ]
  [0.05886333 0.        ]]

 [[0.05886333 0.        ]
  [0.05886333 0.        ]
  [0.05886333 0.        ]
  [0.05886333 0.        ]]]


# ViT

上記を順番に行うだけ。

最後にパッチ分割した分の配列の変化を元に戻して、和を取る。

In [30]:
class SimpleViT(nn.Module):
  def __init__(self, patch_size, depth, n_heads):
    #
    # n_patches:サイトをパッチで分割したときのパッチの数
    #
    super().__init__()

    self.n_pathces = n_site//patch_size

    self.Patching = Patching(patch_size)

    self.Encoder = Encoder(depth, n_heads)

    self.output_concat = Rearrange('s np ps -> s (np ps)', np = self.n_pathces, ps = patch_size)

  def forward(self, x):

    output = self.Patching(x)
    output = self.Encoder(output)
    output = self.output_concat(output)
    output = output.sum(axis=1)
    return output
  def forward_cpu(self, x):
    output = torch.from_numpy(x.astype(np.float32)).detach().to(device)
    with torch.no_grad():
      output = self.forward(output)
    return output.to('cpu').detach().numpy().copy()

In [31]:
n_sample = 10
n_site = 16

print('---check---')
if n_site % patch_size == 0:
  print('number of patches: ', n_site//patch_size)
else:
  print('patch size: NG')

if patch_size % n_heads == 0:
  print('dimension per head: ', patch_size//n_heads)
else:
  print('number of heads: NG')

---check---
number of patches:  8
dimension per head:  1


In [32]:
num = np.random.rand(n_sample, n_site)
print(num)
model = SimpleViT(patch_size=patch_size, depth=depth, n_heads=n_heads)
model = model.to(device)
output = model.forward_cpu(num)
print(output.shape)
print(output)

[[0.87816168 0.41624672 0.48155183 0.78003703 0.17704251 0.87117052
  0.20424827 0.03577197 0.44510434 0.46343343 0.0723286  0.8872842
  0.95904707 0.70279473 0.81886196 0.32598124]
 [0.72262004 0.02262676 0.74661716 0.44918226 0.4250344  0.72004776
  0.76506545 0.80493129 0.49840751 0.89695819 0.81188344 0.39618581
  0.19586283 0.49521393 0.47046149 0.23265885]
 [0.00376722 0.83925521 0.66545875 0.87280518 0.48819191 0.35069221
  0.46086927 0.09706341 0.84181032 0.60396023 0.69491252 0.80531268
  0.76083781 0.06311449 0.12271661 0.60432016]
 [0.58882927 0.5511036  0.98911951 0.55643325 0.47431307 0.2937042
  0.14305036 0.03704857 0.77399822 0.03706575 0.97072502 0.62036086
  0.74600643 0.73861576 0.91583704 0.38125537]
 [0.53606336 0.41859981 0.17305056 0.2441288  0.13758955 0.44218887
  0.81361087 0.06360193 0.01875181 0.53764703 0.94714106 0.61738377
  0.28533387 0.78479613 0.28682383 0.58152511]
 [0.01015564 0.1843535  0.22468013 0.14565312 0.36030343 0.28948403
  0.49135017 0.1914